In [1]:
!pip install bokeh geopy tqdm --quiet



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from tqdm import tqdm
output_notebook()


Loading BokehJS ...

In [3]:
df = pd.read_csv('dataset.csv')
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
df.rename(columns={
    'SystemCodeNumber': 'lot_id',
    'Capacity': 'capacity',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Occupancy': 'occupancy',
    'VehicleType': 'vehicle_type',
    'TrafficConditionNearby': 'traffic',
    'QueueLength': 'queue_length',
    'IsSpecialDay': 'is_special_day'
}, inplace=True)
df.fillna(0, inplace=True)
scaler = MinMaxScaler()
df[['occupancy', 'queue_length']] = scaler.fit_transform(df[['occupancy', 'queue_length']])
df = df.sort_values(by='timestamp').reset_index(drop=True)

In [4]:
BASE_PRICE = 10
ALPHA = 5
λ = 0.5
α, β, γ, δ, ε = 1.5, 1.0, 1.2, 2.0, 0.8
vehicle_weight = {'car': 1.0, 'bike': 0.7, 'truck': 1.3, 'cycle': 0.5}
traffic_map = {'low': 0.3, 'medium': 0.6, 'high': 1.0}


In [5]:
df['price_linear'] = float(BASE_PRICE)
for i in range(1, len(df)):
    if df.loc[i, 'lot_id'] == df.loc[i - 1, 'lot_id']:
        occ = df.loc[i, 'occupancy']
        prev_price = df.loc[i - 1, 'price_linear']
        new_price = prev_price + ALPHA * occ
        df.loc[i, 'price_linear'] = np.clip(new_price, BASE_PRICE * 0.5, BASE_PRICE * 2)


In [6]:
def demand_function(row):
    traffic_score = traffic_map.get(str(row['traffic']).lower(), 0.5)
    vehicle_score = vehicle_weight.get(row['vehicle_type'], 1.0)
    return (
        α * row['occupancy'] +
        β * row['queue_length'] -
        γ * traffic_score +
        δ * row['is_special_day'] +
        ε * vehicle_score
    )
df['demand'] = df.apply(demand_function, axis=1)
df['demand_norm'] = (df['demand'] - df['demand'].min()) / (df['demand'].max() - df['demand'].min())
df['price_demand'] = BASE_PRICE * (1 + λ * df['demand_norm'])
df['price_demand'] = df['price_demand'].clip(BASE_PRICE * 0.5, BASE_PRICE * 2)

In [ ]:
price_competitive_list = []
for ts in tqdm(df['timestamp'].unique(), desc="Calculating competitive prices"):
    ts_df = df[df['timestamp'] == ts]
    for _, row in ts_df.iterrows():
        current_coords = (row['latitude'], row['longitude'])
        competitors = ts_df[ts_df['lot_id'] != row['lot_id']]
        nearby_prices = [
            comp['price_demand'] for _, comp in competitors.iterrows()
            if geodesic(current_coords, (comp['latitude'], comp['longitude'])).km < 0.5
        ]
        if nearby_prices:
            avg_price = np.mean(nearby_prices)
            if row['occupancy'] > 0.9:
                updated_price = max(BASE_PRICE * 0.5, avg_price * 0.9)
            else:
                updated_price = min(BASE_PRICE * 2, avg_price * 1.1)
        else:
            updated_price = row['price_demand']
        price_competitive_list.append(updated_price)
df['price_competitive'] = price_competitive_list

Calculating competitive prices:  65%|██████████████████████████████▋                | 944/1445 [02:09<01:12,  6.95it/s]

In [ ]:
def plot_prices(lot_id):
    subset = df[df['lot_id'] == lot_id]
    p = figure(title=f"Pricing Trend for Lot {lot_id}", x_axis_type='datetime', width=800)
    p.line(subset['timestamp'], subset['price_linear'], color='blue', legend_label='Linear Model')
    p.line(subset['timestamp'], subset['price_demand'], color='green', legend_label='Demand Model')
    p.line(subset['timestamp'], subset['price_competitive'], color='red', legend_label='Competitive Model')
    p.legend.location = 'top_left'
    show(p)

# Example visualization
plot_prices('BHMBCCMKT01')

In [ ]:
final_output = df[['timestamp', 'lot_id', 'occupancy', 'queue_length', 
                   'price_linear', 'price_demand', 'price_competitive']]

# Display nicely styled output
styled_output = final_output.tail(20).style\
    .background_gradient(cmap='YlGnBu', subset=['price_linear', 'price_demand', 'price_competitive'])\
    .set_caption("📊 Final Dynamic Pricing Results (Last 20 Records)")
styled_output
